#### **Importing AWS Glue Stuff**

In [ ]:
# glue libraries and settings from guide notebook

# %idle_timeout 2880
# %glue_version 4.0
# %worker_type G.1X
# %number_of_workers 5

# import sys
# from awsglue.transforms import *
# from awsglue.utils import getResolvedOptions
# from pyspark.context import SparkContext
# from awsglue.context import GlueContext
# from awsglue.job import Job
  
# sc = SparkContext.getOrCreate()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)

#### **Import Data**

In [1]:
# import python libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for local
df = pd.read_csv('./datasets/airline.csv', encoding = "ISO-8859-1")
holiday = pd.read_csv('./datasets/dim_date_proposed.csv')

# # for running on AWS Glue
# airline_dyf = glueContext.create_dynamic_frame.from_catalog(database='facts_database', table_name='airline.csv')
# df = airline_dyf.toDF()

# holiday_dyf = glueContext.create_dynamic_frame.from_catalog(database='facts_database', table_name='dim_date_proposed_csv')
# # holiday = holiday_dyf.toDF()


### **Data Cleaning**

#### **holiday.csv dataset**

In [4]:
#remove non-holidays
holiday_cleaned = holiday[holiday['holiday_us'].notnull()]

#include only holidays we need in the years in airline.csv (1987 - 2008)
holiday_cleaned = holiday_cleaned[(holiday_cleaned['cal_year'] >= 1987) & (holiday_cleaned['cal_year'] <= 2008)]

#remove unncessary columns
holiday_cleaned = holiday_cleaned[['day_dt','holiday_us']]

#add year column
holiday_cleaned["year"] = holiday_cleaned["day_dt"].str[-4:]
holiday_cleaned["year"].astype(int)

#change the format of day_dt column from mm/dd/yyyy to dd/mm/yyyy
holiday_cleaned['day_dt'] = pd.to_datetime(holiday_cleaned['day_dt'], dayfirst=True)

holiday_cleaned.head(5)

C:\Users\zhenx\AppData\Local\Temp\ipykernel_27016\4087044726.py:15: UserWarning: Parsing '11/26/1987' in MM/DD/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  holiday_cleaned['day_dt'] = pd.to_datetime(holiday_cleaned['day_dt'], dayfirst=True)
C:\Users\zhenx\AppData\Local\Temp\ipykernel_27016\4087044726.py:15: UserWarning: Parsing '12/24/1987' in MM/DD/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  holiday_cleaned['day_dt'] = pd.to_datetime(holiday_cleaned['day_dt'], dayfirst=True)
C:\Users\zhenx\AppData\Local\Temp\ipykernel_27016\4087044726.py:15: UserWarning: Parsing '12/25/1987' in MM/DD/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  holiday_cleaned['day_dt'] = pd.to_datetime(holiday_cleaned['day_dt'], dayfirst=True)
C:\Users\zhenx\AppData\Local\Temp\ipykernel_27016\4087044726.py:15: UserWarning: Parsing '12/31/1987' in MM/DD/YYYY format. Provide form

,day_dt,holiday_us,year
14610,1987-01-01,New Years Day,1987
14628,1987-01-19,MLK Day,1987
14656,1987-02-16,Presidents Day,1987
14754,1987-05-25,Memorial Day,1987
14794,1987-04-07,Independence Day,1987


#### **airline.csv dataset**

In [13]:
#create full_date column
df1 = df
df1["full_date"] = df["DayofMonth"].astype(str) + "/" + df["Month"].astype(str) + "/" + df["Year"].astype(str)
df1["full_date"] = pd.to_datetime(df["full_date"], format = "%d/%m/%Y", dayfirst=True) #this doesn't change the date properly for some reason, but it works well enough to be able to work w the data

#create is_holiday column
df1["is_holiday"] = df1["full_date"].isin(holiday_cleaned["day_dt"])

#remove rows with ArrTime > 2400
df1 = df1[(df1['ArrTime'] <= 2400)]
df1 = df1[(df1['DepTime'] <= 2400)]

df1.head(5)

df2 = df1.drop(columns=["ActualElapsedTime", "AirTime", "ArrTime", "CRSArrTime", "CRSDepTime", "CRSElapsedTime", "CancellationCode", "FlightNum", "TailNum","TaxiIn", "TaxiOut","UniqueCarrier"])

# one more column for number of flights in that day
df2
df2["NumFlightsToday"] = df2.groupby(["full_date"])["full_date"].transform('count')

MemoryError: Unable to allocate 1.08 GiB for an array with shape (15, 9674807) and data type float64

In [14]:
df2

,ArrDelay,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,WeatherDelay,Year,full_date,is_holiday,NumFlightsToday
0,90.0,0,0.0,3,21,81.0,1616.0,DFW,802.0,0,67.0,6,23.0,ORD,0.0,0.0,2006,2006-06-21,False,560760
1,59.0,0,NaN,2,16,14.0,1524.0,MSP,528.0,0,NaN,9,NaN,DTW,NaN,NaN,1997,1997-09-16,False,423929
3,80.0,0,NaN,2,5,70.0,2210.0,DTW,229.0,0,NaN,2,NaN,MDW,NaN,NaN,1991,1991-02-05,False,402346
4,5.0,0,NaN,6,1,20.0,1800.0,FLL,992.0,0,NaN,2,NaN,PHL,NaN,NaN,1997,1997-02-01,False,423929
5,10.0,0,NaN,2,21,0.0,1300.0,CLE,475.0,0,NaN,2,NaN,BDL,NaN,NaN,1989,1989-02-21,False,395876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9882792,5.0,0,NaN,4,2,10.0,1730.0,PIT,201.0,0,NaN,10,NaN,DTW,NaN,NaN,2008,2008-10-02,False,549232
9882793,5.0,0,NaN,4,25,9.0,811.0,DEN,641.0,0,NaN,12,NaN,DFW,NaN,NaN,1997,1997-12-25,True,423929
9882794,-10.0,0,NaN,7,30,1.0,551.0,DFW,984.0,0,NaN,8,NaN,MCO,NaN,NaN,1992,1992-08-30,False,402151
9882795,-4.0,0,NaN,4,3,0.0,620.0,SMF,389.0,0,NaN,5,NaN,ONT,NaN,NaN,2001,2001-05-03,False,457210


In [ ]:
df1.to_csv("clean_airline.csv")
holiday_cleaned.to_csv("clean_holiday.csv")

#### **Export to S3**

In [ ]:
s3output = glueContext.getSink(
  path="s3://factors-cleaned-data-bucket",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setFormat("glueparquet")
s3output.writeFrame(DyF)

# **Data Visualization**

### Line Graph (NEW)

In [ ]:
def day_of_year_list(timestamps):
    day_of_year_list = [timestamp.dayofyear for timestamp in timestamps]
    day_of_year_list = [day - 1 for day in day_of_year_list]
    return day_of_year_list

#creating  line graph
df_year = 1999

plot_data = df1.groupby("full_date")["full_date"].count()
plot_df = pd.DataFrame({'full_date':plot_data.index,'flight_count':plot_data.values})
plot_df_1999 = plot_df[plot_df["full_date"].dt.year == df_year]

x_axis = plot_df_1999["full_date"]
y_axis = plot_df_1999["flight_count"]

#creating holiday markers
holiday1 = holiday[holiday["cal_year"] == df_year]
holiday2 = holiday1[["holiday_us", "day_of_year"]]
holiday_label_list = []
holiday_labels = []
for index, row in holiday2.iterrows():
    if pd.isna(row["holiday_us"]):
        holiday_label_list.append("")
    else:
        holiday_label_list.append(row["holiday_us"])
        holiday_labels.append(row["day_of_year"]-1)

plt.figure(figsize=(25, 5))
plt.plot(x_axis, y_axis,  marker="o", markevery=holiday_labels, mfc = "r")
for x, y, text in zip(x_axis, y_axis, holiday_label_list):
    plt.text(x, y, text)

plt.xticks(rotation=50, ha="right")
plt.show()


### Analysis

In [ ]:

from datetime import timedelta

def get_week_dates(timestamps):
    holiday_dates = {}
    for timestamp in timestamps:
        holiday_dates[timestamp] = []
        holiday_dates[timestamp].append(timestamp)
        for i in range(1,8):
            i_days_before = timestamp - timedelta(days=i)
            i_days_after = timestamp + timedelta(days=i)
            holiday_dates[timestamp].append(i_days_before)
            holiday_dates[timestamp].append(i_days_after)
    return holiday_dates

def remove_duplicate(timestamps):
    timestamp_list = []
    for timestamp in timestamps:
        if timestamp not in timestamp_list:
            timestamp_list.append(timestamp)
    return timestamp_list

                                                                                                                                                                            

result_df = pd.DataFrame(columns=["year","holiday w/ highest effect","diff_high", "holiday w/ worst effect","diff_low"])

year_list = list(range(1988,2008))
for year in year_list:
    plot_df_year = plot_df[plot_df["full_date"].dt.year == year]
    holiday1 = holiday[holiday["cal_year"] == year]

    analysis_df1 = holiday1[["day_dt","holiday_us"]][holiday1[["day_dt","holiday_us"]]["holiday_us"].notna()]
    analysis_df2 =  pd.to_datetime(analysis_df1['day_dt'],dayfirst=True)
    analysis_df2_list = analysis_df2.to_list()

    analysis_df2_dict = get_week_dates(analysis_df2_list)

    analysis_df3 = pd.DataFrame(columns=['holiday_date','flight_count_average_2_weeks','max','min'])

    for holidayx, date_list in analysis_df2_dict.items():
        holiday_flight_total = 0
        holiday_max = 0
        holiday_min = 1000000
        for date2 in date_list:
            day_flight_total = plot_df["flight_count"][plot_df["full_date"]== date2]
            day_flight_total = day_flight_total.iloc[0]
            holiday_flight_total += day_flight_total
            if day_flight_total > holiday_max:
                holiday_max = day_flight_total
            if day_flight_total < holiday_min:
                holiday_min = day_flight_total
            flight_count_average = holiday_flight_total/15
            flight_count_average = flight_count_average.round()
        temp_df = pd.DataFrame({'holiday_date':[holidayx],'flight_count_average_2_weeks':[flight_count_average], 'max':[holiday_max], 'min':[holiday_min]})
        analysis_df3 = pd.concat([analysis_df3,temp_df])
        

    diff_column = pd.DataFrame(columns=['diff_from_avg'])
    for index, row in analysis_df3.iterrows():
        diff_from_avg = row[1] - plot_df_year["flight_count"].mean()
        temp_df = pd.DataFrame({'diff_from_avg':[diff_from_avg]})
        diff_column = pd.concat([diff_column,temp_df])

    analysis_df3 = pd.concat([analysis_df3, diff_column], axis=1)

    highest_holiday = ""
    lowest_holiday = ""
    highest_holiday_diff = 0
    lowest_holiday_diff = 100000
    for index, row in analysis_df3.iterrows():
        diff = row[4]
        holiday_date = row[0]
        if diff < lowest_holiday_diff:
            lowest_holiday_diff = diff
            lowest_holiday = holiday_date
        if diff > highest_holiday_diff:
            highest_holiday_diff = diff
            highest_holiday = holiday_date

    # print("Mean of all daily flights for year " + str(year) + ": " + str(plot_df_year["flight_count"].mean()))
    # print("Min of all daily flights for year " + str(year) + ": " + str(plot_df_year["flight_count"].min()))
    # print("Max of all daily flights for year " + str(year) + ": " + str(plot_df_year["flight_count"].max()))

    temp_df = pd.DataFrame({'year':[year],'holiday w/ highest effect':[highest_holiday],'diff_high':[highest_holiday_diff], 'holiday w/ worst effect':[lowest_holiday], "diff_low":[lowest_holiday_diff]})
    result_df = pd.concat([result_df,temp_df])



In [ ]:
result_df1 = result_df
result_df1 = result_df1.reset_index(drop=True)
for index, row in result_df1.iterrows():
    holiday_name_high = holiday_cleaned.loc[holiday_cleaned["day_dt"]==row[1], "holiday_us"].iloc[0]
    result_df1.at[index, 'holiday w/ highest effect'] = holiday_name_high
    holiday_name_low = holiday_cleaned.loc[holiday_cleaned["day_dt"]==row[3], "holiday_us"].iloc[0]
    result_df1.at[index, 'holiday w/ worst effect'] = holiday_name_low
result_df1
result_df1['holiday w/ worst effect'].value_counts()


result_df2 = result_df1['holiday w/ worst effect'].value_counts().rename_axis("holiday w/ worst effect").reset_index(name="count")
high_holiday_list = result_df2["holiday w/ worst effect"].to_list()
high_count_list = result_df2["count"].to_list()

y = np.array(high_count_list)
mylabels = high_holiday_list

plt.pie(y, labels = mylabels)
plt.show() 

## Line Graph (OLD)


In [ ]:
# #function for markers
# def day_of_year(date_str):
#     month, day, year = map(int, date_str.split('/'))
#     days_in_month = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
#     if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
#         days_in_month[2] += 1  # Leap year adjustment
#     return days_in_month[month - 1] + day - 1  # Subtract 1 to convert to 0-based indexing

                                                                                                                                                                                                 

# year_to_visualize = 1999
# year_to_visualize_string = "1999"
# df_year = df1[(df1['Year'] == year_to_visualize)]

# flights_total = df_year.groupby('full_date').size()

# holidays = df_year[df_year['is_holiday'] == True]
# holidays = holidays.groupby('full_date').size()

# holiday_df = holidays.to_frame()
# holiday_df.reset_index(inplace=True)

# holiday_list = holiday_df['full_date'].tolist()
# print(holiday_list)
# day_of_year_list = [day_of_year(date_str) for date_str in holiday_list]
# # print(day_of_year_list)

# plt.figure(figsize=(20, 12))
# flights_total.plot(kind='line', marker='o', mfc = "r", color='blue', linestyle='-',markevery=day_of_year_list)

# plt.xlabel('Time')
# plt.ylabel('Number of Flights')
# plt.title('Number of Domestic Flights from 1987 to 2008 with Holidays Highlighted')
# plt.grid(True)
# plt.legend(['Number of Flights', 'Holiday'])

# plt.show()